<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/codecompletion_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
try:
  import google.colab
  DOCKER = False
except:
  DOCKER = True
print(DOCKER)

True


In [7]:
# We could modify these paths to "stub" behavior for test/dev
workspaceDir = "/content"
GPTNeoXColabDirName = "GPT-NeoX-Colab"
if DOCKER:
    GPTNeoXColabDir = f"/workspace"
else:
    GPTNeoXColabDir = f"{workspaceDir}/{GPTNeoXColabDirName}"

In [9]:
%%time
#@title Clone GPT-NeoX-Colab
if DOCKER:
    %cd {GPTNeoXColabDir}
else:
    %cd {workspaceDir}
    # Don't use --depth 1 because that does not play nice with git-annex
    !git clone https://github.com/markNZed/GPT-NeoX-Colab.git
    %cd {GPTNeoXColabDir}
    %pip install -q -r requirements_colab.txt
    %pip install -q .

/workspace
CPU times: user 440 μs, sys: 2.84 ms, total: 3.28 ms
Wall time: 2.64 ms


In [10]:
%cd {GPTNeoXColabDir}
from dotenv import load_dotenv
import os
load_dotenv(f"{GPTNeoXColabDir}/.env")
import GPTNeoXColab
GPTNeoXColab.utils.colab.fetch_data("data/codecompletion/token_completion.tar.gz")
%cd {GPTNeoXColabDir}/data/codecompletion
if not os.path.exists(f"data/codecompletion/token_completion"):
    !tar -xzf token_completion.tar.gz
%cd {GPTNeoXColabDir}
GPTNeoXColab.utils.colab.fetch_data("models/codecompletion/global_step7000_HF.tar.gz")
%cd {GPTNeoXColabDir}/models/codecompletion
if not os.path.exists(f"latest"):
    !tar -xzf global_step7000_HF.tar.gz
    !mv global_step7000_HF latest

/workspace


Data retrieval successful.
/workspace/data/codecompletion
/workspace
Data retrieval successful.
/workspace/models/codecompletion


# Using Byte-Pair Encoding Tokenizer

In [11]:
%cd {GPTNeoXColabDir}/models/codecompletion/latest
if not os.path.exists("gpt2-vocab.json"):
    !wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json
    !mv gpt2-vocab.json vocab.json
if not os.path.exists("gpt2-merges.txt"):
    !wget https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt
    !mv gpt2-merges.txt merges.txt

/workspace/models/codecompletion/latest
--2024-11-15 22:30:55--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.15.193.188, 52.217.129.248, 52.217.1.198, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.15.193.188|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving to: ‘gpt2-vocab.json’

gpt2-vocab.json     100%[===================>]   1018K   791KB/s    in 1.3s    

2024-11-15 22:30:57 (791 KB/s) - ‘gpt2-vocab.json’ saved [1042301/1042301]

--2024-11-15 22:30:57--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.12.150, 52.217.91.182, 52.217.168.144, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.12.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘gpt2-merges.txt’

gpt2-merges.txt     10

# HuggingFace Inference

In [12]:
from transformers import GPTNeoXForCausalLM, GPT2Tokenizer
import torch

%cd {workspaceDir}

# Initialize the tokenizer with your vocabulary and merge files
tokenizer = GPT2Tokenizer(vocab_file=f"{GPTNeoXColabDir}/models/codecompletion/latest/vocab.json", merges_file=f"{GPTNeoXColabDir}/models/codecompletion/latest/merges.txt")

# Load your model
model_path = f"{GPTNeoXColabDir}/models/codecompletion/latest"
model = GPTNeoXForCausalLM.from_pretrained(model_path)

# Set the model to evaluation mode
model.eval()

# Prompt the user for input
input_text = """<s> import sys , os <EOL> import imp <EOL> from optparse import make_option <EOL> from django . conf import settings <EOL> from django"""

# Tokenize and prepare input
input_ids = torch.tensor([tokenizer.encode(input_text)], dtype=torch.long)
attention_mask = torch.ones_like(input_ids)  # Create an attention mask for non-padded input

# Generate text with specified pad_token_id and attention_mask
with torch.no_grad():
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=200,          # Adjust this for desired output length
        temperature=0.7,        # Controls creativity
        top_k=50,               # Controls diversity
        top_p=0.9,              # Nucleus sampling
        num_return_sequences=1, # Number of sequences to return
        pad_token_id=model.config.eos_token_id,  # Set pad_token_id explicitly
        do_sample=True           # Enable sampling mode to use temperature and top_p
    )

# Decode the generated text
generated_text = tokenizer.decode(output[0].tolist())
print("Generated text:", generated_text)

# Function to replace special tokens with original representations
def replace_special_tokens(text):
    """
    Replaces special tokens in the generated text with more readable or context-appropriate representations.
    """
    replacements = {
        "<EOL>": "\n",          # Replace with actual newline for code formatting
        "<s>": "",              # Remove start token as it's not necessary in final output
        "</s>": "",             # Remove end token as it's not necessary in final output
        "<pad>": "",            # Remove padding tokens
        "<|UNKNOWN|>": "[UNK]", # Represent unknown tokens in a readable way
        "<STR_LIT>": "\"STRING_LITERAL\"",  # Placeholder for string literals
        "<NUM_LIT>": "0",       # Placeholder for numeric literals
        "<BOOL_LIT>": "True",   # Placeholder for boolean literals (e.g., True/False)
        "<COMMENT>": "# COMMENT",  # Placeholder for comments in the code
    }

    # Replace each special token in text with its corresponding value in `replacements`
    for token, replacement in replacements.items():
        text = text.replace(token, replacement)

    return text.strip()  # Strip leading/trailing whitespace for clean output

# Replace special tokens in the generated text
final_text = replace_special_tokens(generated_text)

# Print the final output
print("Final text:", final_text)


/content
Generated text: <s> import sys , os <EOL> import imp <EOL> from optparse import make_option <EOL> from django . conf import settings <EOL> from django . conf . urls import url <EOL> from django . conf import settings <EOL> from django . utils import six <EOL> from django . utils import six <EOL> from django . utils import six <EOL> from django . utils import six <EOL> from django . utils import six <EOL> from django . utils import six <EOL> from django . utils import six <EOL> from django . utils import six <EOL> from django . utils import six <EOL> from django . utils import six <EOL> from django . utils import six <EOL> from django .
Final text: import sys , os 
 import imp 
 from optparse import make_option 
 from django . conf import settings 
 from django . conf . urls import url 
 from django . conf import settings 
 from django . utils import six 
 from django . utils import six 
 from django . utils import six 
 from django . utils import six 
 from django . utils impo